# A Step-by-Step Guide to Principal Component Analysis

Principal Component Analysis (PCA) is an important unsupervised learning algorithm used for dimensionaly reduction and measurement of redundany in data. The maths behind PCA involves eigenvalues, eigenvectors, eigen-decomposition and other elaborate concepts. This poses a challenge especially to those students who could not yet take an introiductory class to linear algebra. In this notebook we implement PCA from scratch for a very simple dataset with only 10 records and 5 features. It is thought to be a step-by-step demystification of PCA. 

© Marc Pouly & Reza Kakooee, Algorithmic Business Research Lab, HSLU

In [ ]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

# The Dataset

In [ ]:
df = pd.DataFrame({
    'Price': {0: 36.1, 1: 15.8, 2: 33.9, 3: 12.5, 4: 23.3, 5: 21.5, 6: 11.8, 7: 29.1, 8: 37.7, 9: 17.5}, 
    'Seats': {0: 8, 1: 4, 2: 6, 3: 4, 4: 6, 5: 6, 6: 4, 7: 6, 8: 6, 9: 4}, 
    'Horsepower': {0: 210, 1: 141, 2: 200, 3: 90, 4: 178, 5: 160, 6: 110, 7: 172, 8: 172, 9: 140}, 
    'Mileage': {0: 1840, 1: 2090, 2: 2335, 3: 3250, 4: 2385, 5: 2045, 6: 2435, 7: 2280, 8: 2535, 9: 2610}, 
    'Passengers': {0: 6, 1: 6, 2: 5, 3: 4, 4: 4, 5: 5, 6: 5, 7: 5, 8: 6, 9: 4}})

df

The dataset contains 10 samples and 5 features with different units. Some features like `Mileage` include large numbers, while some features like `Seats` and `Passengers` have small values. Let us look deeper into the stats:

In [ ]:
df.describe()

The features have `non-zero mean`, and most of them have `std > 1`. For applying `PCA`, we first need to standardize the data such that each feature has `zero mean` and `unit variance`.

# Standardizing Features

In [ ]:
# Transform data such that mean becomes 0
df = df - df.mean()

df

In [ ]:
# Transform data such that standard deviation becomes 1
df = df / df.std()

df

In [ ]:
# Verify the stats
df.describe()

Now all features have `zero mean` and `unit variance (std)`.

# The Covariance Matrix of $X$

The idea of PCA is to find a transformation matrix `P` that transforms the dataset `X` into another dataset `Y = PX` such that the covariance matrix of `Y` is a diagonal matrix. In a diagonal matrix, all off-diagonal elements are zero, which means that we removed all redundancy among features. Let us look at the covariance matrix of `X`

In [ ]:
# Covariance matrix calculateed with pandas
covX = df.cov()

sns.heatmap(covX, xticklabels=covX.columns, yticklabels=covX.columns, annot=True)

As expected, off-diagonal elements are not zero. We obtain the same result (up to a constant) by calculating `XX^T`

In [ ]:
# Covariance matrix calculated by XX^T
X = df.values.T
D = X @ X.T

sns.heatmap(1/(len(df)-1) * D, xticklabels=covX.columns, yticklabels=covX.columns, annot=True)

# Extract  `Eigenvalues` and  `Eigenvectors` of $X X^T$

The PCA transformation is obtained if we set $P^T = E$, where the columns of $E$ are the eigenvectors of $X X^T$.
We therefore compute the eigenvectors of $D = X X^T$ with a linear algebra Python package

In [ ]:
# eigs are eigenvalues, E are eigenvectors
eigs, E = scipy.linalg.eigh(D)

eigs

In [ ]:
## Sort the eigenvalues and eigenvectors array in decending order with respect to eigenvalues
idx = eigs.argsort()[::-1]   
eigs = eigs[idx]
E = E[:,idx]

eigs

In [ ]:
E

# Build the Transformation Matrix 

The transformation matrix P is made from the eigenvectors of $D$

In [ ]:
P = E.T

# Transform the Data

In [ ]:
Y = (P @ X).T

## Convert Y back into a dataframe
Y_df = pd.DataFrame(Y, columns=[f'PC_{i+1}' for i in range(Y.shape[1])])

Y_df

Let us see if the covariance matrix is really a diagonal matrix

In [ ]:
covY = Y_df.cov()

sns.heatmap(covY, xticklabels=covX.columns, yticklabels=covX.columns, annot=True)

Note, the values off the diagonal are zero; $10^{-16}$ is super close to zero; this is a phenomenon of numeric rounding and finite precision on computers. 

# Variance Ratio

For dimensionality reduction, we are interested in those principal components with the highest variance (= eigenvalue). Let us calculate the variance ratio to check how many principal components we need to keep. 

In [ ]:
explained_variance_ratio_ = eigs / sum(eigs)

explained_variance_ratio_

For example, the first dimension (eigenvector) explains 70% of the variance in the data. Here is the same information as plot

In [ ]:
fig = plt.figure()
xx = np.array(list(range(len(explained_variance_ratio_))))+1
plt.plot(xx, explained_variance_ratio_, '-bo')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.show()

In [ ]:
# Let us keep only the first two ones, as seems they are much bigger than the rest.
K = 2

info_ratio = sum(explained_variance_ratio_[0:K])/sum(explained_variance_ratio_)*100

print(f"The first two principal components contain about {info_ratio:.2f}% of the whole information.")

# Projection to 2D

We drop `PC_3`, `PC_4`, and `PC_5` from the `Y_df` and visualize the data projected to 2D.

In [ ]:
target_df = Y_df.copy()

for col in ['PC_3', 'PC_4', 'PC_5']:
    target_df = target_df.drop(col, axis=1)
    
target_df

In [ ]:
fig = px.scatter(target_df, x="PC_1", y="PC_2")
fig.update_layout(width = 500, height = 500)
fig.show()

Observe that the data is spread out widely along the x-axis, and less along the y-axis, emphasizing the importance of the 1st principal component over the 2nd. 

# Verification

Let us verify our results by comparing with the `scikit-learn` implementation of PCA:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(df)

print(pca.explained_variance_ratio_)

In [ ]:
PC_values = np.arange(pca.n_components_) + 1

plt.plot(PC_values, pca.explained_variance_ratio_, 'ro-', linewidth=2)
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.show()

`Scikit-Learn` results are consistent with our calculations.

We hope we could successfully demystify PCA. It is not that difficult :-) 

Cheers

Marc & Reza